In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

# Load the file (use the exact file path shown in the Files tab)
data = np.load('/content/drive/MyDrive/17_data.npy')

# Display the shape of the data to ensure it loaded correctly
print(data.shape)


(80000, 36, 1000)


In [ ]:
# Split into real and imaginary parts
real_part = np.real(data).astype(np.float32)
imaginary_part = np.imag(data).astype(np.float32)

# Check the dtype of the real and imaginary parts
print(real_part.dtype)        # Should output float32
print(imaginary_part.dtype)   # Should output float32

# Verify the shape remains unchanged
print(real_part.shape)
print(imaginary_part.shape)


float32
float32
(80000, 36, 1000)
(80000, 36, 1000)


In [ ]:
labels = np.load('/content/drive/MyDrive/17_labels.npy')

In [ ]:
X = np.stack([real_part, imaginary_part], axis=-1)

In [ ]:
print(X.shape, labels.shape)

(80000, 36, 1000, 2) (80000,)


In [ ]:
from sklearn.model_selection import train_test_split

# Split into training and validation sets (80-20 split)
X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.2, random_state=42)

# Check the shapes
print(X_train.shape, X_val.shape)
print(y_train.shape, y_val.shape)



(64000, 36, 1000, 2) (16000, 36, 1000, 2)
(64000,) (16000,)


In [ ]:
def build_model(hp):
    model = models.Sequential()

    # Adjust input shape for Conv2D
    model.add(layers.Conv2D(filters=hp.Int('filters', min_value=16, max_value=64, step=16),
                             kernel_size=(3, 3), activation='relu',
                             input_shape=(X_train.shape[1], X_train.shape[2], 2)))  # Updated for 2D input
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(units=hp.Int('units', min_value=32, max_value=128, step=32), activation='relu'))

    model.add(layers.Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model


In [ ]:
!pip install keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from keras_tuner import Hyperband

# Custom callback to stop tuning
class CustomStopCallback(tf.keras.callbacks.Callback):
    def __init__(self, tuner):
        super().__init__()
        self.tuner = tuner

    def on_epoch_end(self, epoch, logs=None):
        # Specify your stopping condition here
        if epoch >= 5:  # Example: stop after 5 epochs
            print("Stopping tuning after 5 epochs.")
            self.model.stop_training = True

# Create the tuner
tuner = Hyperband(build_model,
                  objective='val_accuracy',
                  max_epochs=10,
                  factor=3,
                  directory='/content/drive/MyDrive/combined/kt_tuning',
                  project_name='combined_17db_binary')

# Search for hyperparameters
tuner.search(X_train, y_train,
             validation_data=(X_val, y_val),
             epochs=10,
             callbacks=[CustomStopCallback(tuner)])  # Pass the custom callback

# Get the best hyperparameters found so far
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Optionally, you can also save the best hyperparameters to a file
best_hps.save('best_hyperparameters.json')


Trial 7 Complete [00h 22m 39s]
val_accuracy: 0.4934374988079071

Best val_accuracy So Far: 0.5071250200271606
Total elapsed time: 02h 17m 01s

Search: Running Trial #8

Value             |Best Value So Far |Hyperparameter
32                |64                |filters
32                |128               |units
0.2               |0.3               |dropout
0.0001            |0.0001            |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
2000/2000 [==============================] - 294s 146ms/step - loss: 39566072.0000 - accuracy: 0.4964 - val_loss: 5073588.0000 - val_accuracy: 0.5073
Epoch 2/2
 467/2000 [======>.......................] - ETA: 3:23 - loss: 5625913.5000 - accuracy: 0.4977

KeyboardInterrupt: 

In [ ]:
# Train the model with the best hyperparameters
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

# Save the best model
model.save('/content/drive/MyDrive/combined/best_combined_17db_model.h5')


In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.show()

# Plot loss
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
import os
import numpy as np

# Create a directory to store layer weights
weights_dir = '/content/drive/MyDrive/combined/layer_weights'
os.makedirs(weights_dir, exist_ok=True)

# Save weights of each layer in the model
for i, layer in enumerate(model.layers):
    weights = layer.get_weights()  # Returns a list of NumPy arrays
    if weights:  # Not all layers (like activation layers) have weights
        for j, weight in enumerate(weights):
            # Save each weight array individually
            np.save(os.path.join(weights_dir, f'layer_{i}_weight_{j}.npy'), weight)
            print(f'Saved weights of layer {i} ({layer.name}), weight {j}')


Saved weights of layer 0 (conv2d_1), weight 0
Saved weights of layer 0 (conv2d_1), weight 1
Saved weights of layer 3 (dense_2), weight 0
Saved weights of layer 3 (dense_2), weight 1
Saved weights of layer 5 (dense_3), weight 0
Saved weights of layer 5 (dense_3), weight 1
